In [3]:
import cv2
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from facenet_pytorch import MTCNN

# Paths
model_save_path = "adaptive_efficientnetv2_rw_s_emotion_model.pth" 

# Emotion categories
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the emotion model
def load_model(model_path):
    model = models.efficientnet_b2(pretrained=True)
    model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.6),
        nn.Linear(model.classifier[1].in_features, len(emotion_classes)),
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

# Load the trained model
model = load_model(model_save_path)

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True, device=device)

# Image preprocessing function
def preprocess_face(face_image):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),          
        transforms.ToTensor(),                   
        transforms.Normalize(mean=[0.485], std=[0.229]),  
    ])
    pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
    processed_image = transform(pil_image)
    processed_image = processed_image.unsqueeze(0) 
    return processed_image.to(device)

# Prediction function
def predict_emotion(face_image):
    # Preprocess the input image
    processed_image = preprocess_face(face_image)

    # Perform prediction
    with torch.no_grad():
        outputs = model(processed_image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        predicted_class = emotion_classes[predicted_class_idx]
        confidence = probabilities[0, predicted_class_idx].item()

    return predicted_class, confidence

# Function to check for quit button clicks
def check_quit_click(event, x, y, flags, param):
    global quit_program
    if event == cv2.EVENT_LBUTTONDOWN:
        if quit_button_coords[0] <= x <= quit_button_coords[2] and quit_button_coords[1] <= y <= quit_button_coords[3]:
            quit_program = True

# Face detection and emotion prediction with OpenCV rendering
def run_live_emotion_detection():
    global quit_program
    quit_program = False

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot access webcam.")
        return

    print("Press 'q' or click 'Quit' button to exit the application.")

    # Define quit button coordinates
    global quit_button_coords
    quit_button_coords = (10, 10, 100, 50)

    # Set mouse callback
    cv2.namedWindow("Live Emotion Detection")
    cv2.setMouseCallback("Live Emotion Detection", check_quit_click)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read frame from webcam.")
            break

        # Convert frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces
        boxes, _ = mtcnn.detect(rgb_frame)

        # Process detected faces
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]
                face_image = frame[y1:y2, x1:x2]

                # Skip prediction if the face is too small
                if face_image.shape[0] > 0 and face_image.shape[1] > 0:
                    emotion, confidence = predict_emotion(face_image)
                    print(f"Emotion Detected: {emotion} (Confidence: {confidence:.2f})")

                    # Draw the rectangle
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Display the emotion label
                    label = f"{emotion} ({confidence:.2f})"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        else:
            cv2.putText(frame, "No face detected", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw the quit button
        cv2.rectangle(frame, (quit_button_coords[0], quit_button_coords[1]), 
                      (quit_button_coords[2], quit_button_coords[3]), (0, 0, 255), -1)
        cv2.putText(frame, "Quit", (quit_button_coords[0] + 10, quit_button_coords[1] + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow("Live Emotion Detection", frame)

        # Check for quit conditions
        if cv2.waitKey(1) & 0xFF == ord('q') or quit_program:
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the emotion detection
if __name__ == "__main__":
    run_live_emotion_detection()


C:\Users\Tuf\AppData\Local\Temp\ipykernel_4800\3292540270.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=devi

Press 'q' or click 'Quit' button to exit the application.
Emotion Detected: Sad (Confidence: 0.23)
Emotion Detected: Neutral (Confidence: 0.35)
Emotion Detected: Neutral (Confidence: 0.35)
Emotion Detected: Sad (Confidence: 0.22)
Emotion Detected: Happy (Confidence: 0.29)
Emotion Detected: Neutral (Confidence: 0.55)
Emotion Detected: Neutral (Confidence: 0.38)
Emotion Detected: Neutral (Confidence: 0.52)
Emotion Detected: Neutral (Confidence: 0.61)
Emotion Detected: Neutral (Confidence: 0.39)
Emotion Detected: Neutral (Confidence: 0.56)
Emotion Detected: Neutral (Confidence: 0.59)
Emotion Detected: Neutral (Confidence: 0.44)
Emotion Detected: Neutral (Confidence: 0.60)
Emotion Detected: Neutral (Confidence: 0.61)
Emotion Detected: Neutral (Confidence: 0.70)
Emotion Detected: Neutral (Confidence: 0.54)
Emotion Detected: Neutral (Confidence: 0.61)
Emotion Detected: Neutral (Confidence: 0.58)
Emotion Detected: Neutral (Confidence: 0.61)
Emotion Detected: Neutral (Confidence: 0.57)
Emotion